In [7]:
from pyspark.sql.types import StringType, StructType, StructField
from pyspark.sql.functions import from_json, col

# Define Event Hub config
connectionString = "Endpoint=sb://srinieventhub.servicebus.windows.net/;SharedAccessKeyName=srinistreaming;SharedAccessKey=W/+ntaPHDnDJb72DHzvUCEXpErhNLRpNE+AEhPgi+nc=;EntityPath=srinieventhubinstance"

eh_conf = {
    'eventhubs.connectionString': sc._jvm.org.apache.spark.eventhubs.EventHubsUtils.encrypt(connectionString)
}

# Optional: Define schema of the JSON message
schema = StructType([
    StructField("messageid", StringType(), True),
    StructField("device_id", StringType(), True),
    StructField("temperature", StringType(), True),
    StructField("humidity", StringType(), True),
    StructField("timestamp", StringType(), True)
])

# Read stream from Event Hub
eventhub_stream = (
    spark.readStream
         .format("eventhubs")
         .options(**eh_conf)
         .load()
)

# Extract message body (which is in binary), convert to string and parse JSON
parsed_stream = (
    eventhub_stream
    .withColumn("body", col("body").cast("string"))
    .withColumn("data", from_json(col("body"), schema))
    .select("data.*")
)

# Write to ADLS Gen2 in Parquet format
query = (
    parsed_stream.writeStream
        .format("parquet")
        .option("checkpointLocation", "abfss://srinicontainer@sriniadls123.dfs.core.windows.net/checkpoints/eventhub/")
        .option("path", "abfss://srinicontainer@sriniadls123.dfs.core.windows.net/ingested/eventhub/")
        .outputMode("append")
        .start()
)
